#**TASK**

## Discription:
1. Role : Quantitive Researcher
2. Department : Commodity trading desk

You need to create a prototype pricing model that can go through further validation and testing before being put into production. Eventually, this model may be the basis for fully automated quoting to clients, but for now, the desk will use it with manual oversight to explore options with the client.

You should write a function that is able to use the data you created previously to price the contract. The client may want to choose multiple dates to inject and withdraw a set amount of gas, so your approach should generalize the explanation from before. Consider all the cash flows involved in the product.

The input parameters that should be taken into account for pricing are:

Injection dates.
Withdrawal dates.
The prices at which the commodity can be purchased/sold on those dates.
The rate at which the gas can be injected/withdrawn.
The maximum volume that can be stored.
Storage costs.
Write a function that takes these inputs and gives back the value of the contract. You can assume there is no transport delay and that interest rates are zero. Market holidays, weekends, and bank holidays need not be accounted for. Test your code by selecting a few sample inputs.


## Importing Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_excel(r"/content/Natural_Gas_Prices.xlsx")
df

,Date,estimated_price
0,2020-10-31,10.096439
1,2020-11-01,10.109376
2,2020-11-02,10.122322
3,2020-11-03,10.135273
4,2020-11-04,10.148226
...,...,...
1426,2024-09-26,11.864399
1427,2024-09-27,11.875091
1428,2024-09-28,11.885900
1429,2024-09-29,11.896823


## user defined values


In [4]:
date_min = df['Date'].min()
date_max = df['Date'].max()

print(f"The date range is from {date_min} to {date_max}")

The date range is from 2020-10-31 00:00:00 to 2024-09-30 00:00:00


In [13]:
import pandas as pd
from datetime import datetime

# ---------------- Load or define your DataFrame ----------------
# df = pd.read_csv("your_file.csv")  # if reading from a file
df['Date'] = pd.to_datetime(df['Date']).dt.date  # Convert to date-only if not already

# ---------------- Parsing Helpers ----------------
def parse_date_list(input_str):
    return [datetime.strptime(date.strip(), "%Y-%m-%d").date() for date in input_str.split(",")]

def parse_float_list(input_str):
    return [float(x.strip()) for x in input_str.split(",")]

# ---------------- Main Pricing Function ----------------
def price_gas_contract_from_df(df):
    min_date = df['Date'].min()
    max_date = df['Date'].max()

    # User Input
    injection_dates = parse_date_list(input("Enter injection dates (comma-separated YYYY-MM-DD): "))
    injection_volumes = parse_float_list(input("Enter injection volumes (same order): "))
    withdrawal_dates = parse_date_list(input("Enter withdrawal dates (comma-separated YYYY-MM-DD): "))
    withdrawal_volumes = parse_float_list(input("Enter withdrawal volumes (same order): "))
    storage_cost_per_unit_per_day = float(input("Enter storage cost per unit per day: "))

    # Input Length Checks
    if len(injection_dates) != len(injection_volumes):
        print("❌ Injection dates and volumes must match in length.")
        return
    if len(withdrawal_dates) != len(withdrawal_volumes):
        print("❌ Withdrawal dates and volumes must match in length.")
        return

    total_value = 0
    print("\n--- Contract Calculation Breakdown ---")

    for i in range(len(withdrawal_dates)):
        inj_date = injection_dates[i]
        wdr_date = withdrawal_dates[i]
        inj_vol = injection_volumes[i]
        wdr_vol = withdrawal_volumes[i]

        # Validation
        if not (min_date <= inj_date <= max_date):
            print(f"❌ Injection date {inj_date} is outside available data range.")
            return
        if not (min_date <= wdr_date <= max_date):
            print(f"❌ Withdrawal date {wdr_date} is outside available data range.")
            return
        if inj_date > wdr_date:
            print(f"❌ Injection date {inj_date} is after withdrawal date {wdr_date}.")
            return
        if inj_vol < wdr_vol:
            print(f"❌ Withdrawal volume {wdr_vol} cannot exceed injected volume {inj_vol}.")
            return

        # Price Lookup
        try:
            buy_price = df.loc[df['Date'] == inj_date, 'estimated_price'].values[0]
            sell_price = df.loc[df['Date'] == wdr_date, 'estimated_price'].values[0]
        except IndexError:
            print(f"❌ Price not found for one of the dates.")
            return

        # Contract Value Calculation
        storage_days = (wdr_date - inj_date).days
        buy_cost = inj_vol * buy_price
        sell_revenue = wdr_vol * sell_price
        storage_cost = storage_days * wdr_vol * storage_cost_per_unit_per_day
        contract_value = sell_revenue - buy_cost - storage_cost
        total_value += contract_value

        # Print Breakdown
        print(f"\n🔹 Pair {i+1}")
        print(f"Injection: {inj_date}, Volume: {inj_vol}, Buy Price: ₹{buy_price:.4f}")
        print(f"Withdrawal: {wdr_date}, Volume: {wdr_vol}, Sell Price: ₹{sell_price:.4f}")
        print(f"Storage: {storage_days} days × ₹{storage_cost_per_unit_per_day} = ₹{storage_cost:.2f}")
        print(f"→ Value for this pair: ₹{contract_value:.2f}")

    print("\n✅ Total Contract Value: ₹", round(total_value, 2))

# ---------------- To run ----------------
# price_gas_contract_from_df(df)


In [16]:
price_gas_contract_from_df(df)

Enter injection dates (comma-separated YYYY-MM-DD): 2021-01-03 , 2021-02-03
Enter injection volumes (same order): 4 , 5
Enter withdrawal dates (comma-separated YYYY-MM-DD): 2022-01-04 , 2022-09-06
Enter withdrawal volumes (same order): 2,3
Enter storage cost per unit per day: 12

--- Contract Calculation Breakdown ---

🔹 Pair 1
Injection: 2021-01-03, Volume: 4.0, Buy Price: ₹10.8074
Withdrawal: 2022-01-04, Volume: 2.0, Sell Price: ₹11.3593
Storage: 366 days × ₹12.0 = ₹8784.00
→ Value for this pair: ₹-8804.51

🔹 Pair 2
Injection: 2021-02-03, Volume: 5.0, Buy Price: ₹10.9454
Withdrawal: 2022-09-06, Volume: 3.0, Sell Price: ₹10.5818
Storage: 580 days × ₹12.0 = ₹20880.00
→ Value for this pair: ₹-20902.98

✅ Total Contract Value: ₹ -29707.49
